# MicroFlow vs TensorFlow Lite

## Setup Environment

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.get_logger().setLevel("WARNING")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

seed = 1
np.random.seed(seed)

## Generate Dataset

In [ ]:
SAMPLES = 500

x = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES).astype(np.float32)
np.random.shuffle(x)
y = np.sin(x).astype(np.float32)
y += 0.1 * np.random.randn(*y.shape)

plt.plot(x, y, '.')
plt.show()

## Run TensorFlow Lite

In [ ]:
x_test = x.copy()

interpreter = tf.lite.Interpreter(model_path="../models/sine.tflite", experimental_op_resolver_type=tf.lite.experimental.OpResolverType.BUILTIN_REF)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

input_scale, input_zero_point = input_details["quantization"]
if (input_scale, input_zero_point) != (0.0, 0):
    x_test = x_test / input_scale + input_zero_point
    x_test = x_test.astype(input_details["dtype"])
    
y_pred = np.empty(x_test.size, dtype=output_details["dtype"])
for i in range(len(x_test)):
    interpreter.set_tensor(input_details["index"], [[x_test[i]]])
    interpreter.invoke()
    y_pred[i] = interpreter.get_tensor(output_details["index"])[0]
    
output_scale, output_zero_point = output_details["quantization"]
if (output_scale, output_zero_point) != (0.0, 0):
    y_pred = y_pred.astype(np.float32)
    y_pred = (y_pred - output_zero_point) * output_scale
    
pd.DataFrame(zip(x, y_pred), columns=["x", "y_pred"]).to_csv("tflite.csv", index=False)
    
plt.plot(x, y, ".", label="Actual values")
plt.plot(x, y_pred, ".", label="TFLite predictions")
plt.legend()
plt.show()

## Run MicroFlow

In [ ]:
! cd ../.. && cargo run --example=microflow-vs-tflite

microflow = pd.read_csv("microflow.csv")

plt.plot(x, y, ".", label="Actual values")
plt.plot(x, microflow.y_pred, ".", label="MicroFlow predictions")
plt.legend()
plt.show()

## Compare Results

In [ ]:
plt.plot(x, y, ".", label="Actual values")
plt.plot(x, y_pred, ".", label="TFLite predictions")
plt.plot(x, microflow.y_pred, ".", label="MicroFlow predictions")
plt.legend()
plt.show()

pd.DataFrame.from_records(
    [[np.square(np.subtract(y, y_pred)).mean()],
    [np.square(np.subtract(y, microflow.y_pred)).mean()]],
    columns=["MSE"],
    index=["TensorFlow Lite", "MicroFlow"]
).round(4)